# Data-X Spring 2019: Homework 7

### Webscraping



In this homework, you will do some exercises with web-scraping.

## Name: Ran Meng

## SID: 3034368718



### Fun with Webscraping & Text manipulation


## 1. Statistics in Presidential Debates

Your first task is to scrape Presidential Debates from the Commission of Presidential Debates website: https://www.debates.org/voter-education/debate-transcripts/

To do this, you are not allowed to manually look up the URLs that you need, instead you have to scrape them. The root url to be scraped is the one listed above, namely: https://www.debates.org/voter-education/debate-transcripts/


1. By using `requests` and `BeautifulSoup` find all the links / URLs on the website that links to transcriptions of **First Presidential Debates** from the years [1988, 1984, 1976, 1960]. In total you should find 4 links / URLs that fulfill this criteria. **Print the urls.**

2. When you have a list of the URLs your task is to create a Data Frame with some statistics (see example of output below):
    1. Scrape the title of each link and use that as the column name in your Data Frame. 
    2. Count how long the transcript of the debate is (as in the number of characters in transcription string). Feel free to include `\` characters in your count, but remove any breakline characters, i.e. `\n`. You will get credit if your count is +/- 10% from our result.
    3. Count how many times the word **war** was used in the different debates. Note that you have to convert the text in a smart way (to not count the word **warranty** for example, but counting **war.**, **war!**, **war,** or **War** etc.
    4. Also scrape the most common used word in the debate, and write how many times it was used. Note that you have to use the same strategy as in C in order to do this.
    
    **Print your final output result.**
    
**Tips:**

___

In order to solve the questions above, it can be useful to work with Regular Expressions and explore methods on strings like `.strip(), .replace(), .find(), .count(), .lower()` etc. Both are very powerful tools to do string processing in Python. To count common words for example I used a `Counter` object and a Regular expression pattern for only words, see example:

```python
    from collections import Counter
    import re

    counts = Counter(re.findall(r"[\w']+", text.lower()))
```

Read more about Regular Expressions here: https://docs.python.org/3/howto/regex.html
    
    
**Example output of all of the answers to Question 1.2:**


![pres_stats_2](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/presidents_stats_2.jpg)



----

.




In [1]:
# your code here
import requests
import bs4 as bs
import numpy as np
import pandas as pd
from collections import Counter
import re
import warnings
warnings.filterwarnings('ignore')
from time import sleep

In [2]:
# Website downloaded successfully
source = requests.get(" https://www.debates.org/voter-education/debate-transcripts/") 
print(source)

<Response [200]>


In [3]:
soup = bs.BeautifulSoup(source.content, features='html.parser')

In [4]:
#print(soup.body) # Inspect html content

In [5]:
# 1.1
links = soup.find_all('a')
FPD_URLs = []

for l in links:
    if 'First' in l.text and 'Presidential Debate' in l.text:
        if '1960' in l.text or '1976' in l.text or '1984' in l.text or '1988' in l.text:
            FPD_URLs.append(l.get('href'))# All presential debates

            
for l in range(len(FPD_URLs)):
    FPD_URLs[l] = "https://www.debates.org" + FPD_URLs[l]
    print(FPD_URLs[l])

print("\nTotal number of URLs: ", len(FPD_URLs))


https://www.debates.org/voter-education/debate-transcripts/september-25-1988-debate-transcript/
https://www.debates.org/voter-education/debate-transcripts/october-7-1984-debate-transcript/
https://www.debates.org/voter-education/debate-transcripts/september-23-1976-debate-transcript/
https://www.debates.org/voter-education/debate-transcripts/september-26-1960-debate-transcript/

Total number of URLs:  4


In [6]:
#1.2.A
column_titles = []
for l in links: 
    if 'First' in l.text and 'Presidential Debate' in l.text:
        if '1960' in l.text or '1976' in l.text or '1984' in l.text or '1988' in l.text:
            column_titles.append(l.text)
column_titles

['September 25, 1988: The First Bush-Dukakis Presidential Debate',
 'October 7, 1984: The First Reagan-Mondale Presidential Debate',
 'September 23, 1976: The First Carter-Ford Presidential Debate',
 'September 26, 1960: The First Kennedy-Nixon Presidential Debate']

In [7]:
#1.2
debate_char_length = []
war_count = []
most_common_w = []
most_common_w_count = []

for l in range(len(FPD_URLs)):
    warcount = 0
    source = requests.get(FPD_URLs[l])
    soup = bs.BeautifulSoup(source.content, features = 'html.parser')
    transcript = [''.join(s.findAll(text=True)) for s in soup.findAll('p')] #get rid of tags
    transcript = [" ".join(transcript)] #merge into list of length 1
    transcript = transcript[0].lower().replace('\n','')
    debate_char_length.append(len(transcript))

    p = re.compile(r'\bwar\b')
    warcount += len(p.findall(transcript))
    war_count.append(warcount)


    words = re.findall('\w+', transcript) #matches the alphanumeric character in the string
    most_common_w.append(Counter(words).most_common(1)[0][0]) #1 returns the most common word and its count, 
#the first 0 access the word and the count, and the second 0 extracts the word
    most_common_w_count.append(Counter(words).most_common(1)[0][1])
    
    
print(debate_char_length)
print(war_count)
print(most_common_w)
print(most_common_w_count)


[87767, 86858, 80872, 61050]
[8, 2, 7, 3]
['the', 'the', 'the', 'the']
[805, 868, 858, 780]


In [8]:
df = pd.DataFrame([debate_char_length, war_count, most_common_w, most_common_w_count])
df.columns = column_titles
df.set_index(pd.Index(['Debate char length', 'war_count', 'most_common_w', 'most_common_w_count']), inplace = True)
df

,"September 25, 1988: The First Bush-Dukakis Presidential Debate","October 7, 1984: The First Reagan-Mondale Presidential Debate","September 23, 1976: The First Carter-Ford Presidential Debate","September 26, 1960: The First Kennedy-Nixon Presidential Debate"
Debate char length,87767,86858,80872,61050
war_count,8,2,7,3
most_common_w,the,the,the,the
most_common_w_count,805,868,858,780


    
## 2. Download and read in specific line from many data sets

Scrape the first 27 data sets from this URL http://people.sc.fsu.edu/~jburkardt/datasets/regression/ (i.e.`x01.txt` - `x27.txt`). Then, save the 5th line in each data set, this should be the name of the data set author (get rid of the `#` symbol, the white spaces and the comma at the end). 

Count how many times (with a Python function) each author is the reference for one of the 27 data sets. Showcase your results, sorted, with the most common author name first and how many times he appeared in data sets. Use a Pandas DataFrame to show your results, see example. **Print your final output result.**

**Example output of the answer for Question 2:**

![author_stats](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/data_authors.png)


In [9]:
source = requests.get(" http://people.sc.fsu.edu/~jburkardt/datasets/regression/") 
print(source)

<Response [200]>


In [10]:
soup = bs.BeautifulSoup(source.content, features='html.parser')
#print(soup.body)

In [11]:
links = soup.find_all('a')
txt_urls = []
count = 0

for l in links:
    if 'txt' in l.text and count < 27:
        txt_urls.append('http://people.sc.fsu.edu/~jburkardt/datasets/regression/' + l.get('href'))
        count +=1

#sanity check
print('Total number of txt files: ', len(txt_urls))
print('\nFirst 3 of txt urls: ', txt_urls[0:2])

Total number of txt files:  27

First 3 of txt urls:  ['http://people.sc.fsu.edu/~jburkardt/datasets/regression/x01.txt', 'http://people.sc.fsu.edu/~jburkardt/datasets/regression/x02.txt']


In [12]:
authors = []

for l in range(len(txt_urls)):
    sleep(1)
    source = requests.get(txt_urls[l])
    soup = bs.BeautifulSoup(source.content, features = 'html.parser')
    author = soup.get_text().split("\n")[4] #author is in the 5th line
    author = re.sub('#', '', author)
    author = re.sub(',','', author)
    author = ' '.join(author.split())#remove extra whitespaces
    authors.append(author)

#Sanity check
print('Number of authors: ', len(authors)) #count multiple authors as one for each dataset, therefore should return 27

Number of authors:  27


In [13]:
df = pd.DataFrame(Counter(authors).most_common(len(authors)))
df.columns = ['Authors', 'Counts']
df.set_index('Authors', inplace = True)
df

,Counts
Authors,
Helmut Spaeth,16
S Chatterjee B Price,3
R J Freund and P D Minton,2
D G Kleinbaum and L L Kupper,2
S C Narula J F Wellington,2
K A Brownlee,1
S Chatterjee and B Price,1
